In [8]:
import json
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import os
from pathlib import Path

In [2]:
MODEL_NAME = "tdrenis/finetuned-bot-detector"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

In [ ]:


DATA_DIR = "data"
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)
    print(f"Mappa létrehozva: {DATA_DIR}")
else:
    print(f"A '{DATA_DIR}' mappa már létezik.")

Mappa létrehozva: data


In [ ]:
#DATA_DIR = "data"
#input_path = os.path.join(DATA_DIR, "comments.json")
with open("comments.json", "r", encoding="utf-8") as f:
    comments = json.load(f)
inputs = tokenizer(comments, padding=True, truncation=True, return_tensors='pt')

FileNotFoundError: [Errno 2] No such file or directory: 'data\\comments.json'

In [4]:
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

In [5]:
labels = ["human", "bot"]  # Az osztályok, lehet, hogy módosítani kell a modell alapján
results = [{"comment": comment, "prediction": labels[pred.item()]} for comment, pred in zip(comments, predictions)]


In [ ]:
with open("data/bot_detection_results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4, ensure_ascii=False)